In [79]:
import os
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

In [2]:
output_path = os.path.join('..', 'data', 'processed')

In [3]:
df = pd.read_csv(os.path.join(output_path, 'Final_df.csv'))

C:\Users\Nico\AppData\Local\Temp\ipykernel_10812\849079931.py:1: DtypeWarning: Columns (8,12,16,24,29,35,37,38,39,40,42,46,47,48,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(output_path, 'Final_df.csv'))


In [4]:
pd.set_option('display.max_row', 111)  # Permet d'afficher un grand nombre de lignes
pd.set_option('display.max_colum', 111)  # Permet d'afficher un grand nombre de colonnes

In [5]:
df.head()

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh,id_vehicule_x,secu1,secu2,secu3,id_usager,senc,catv,occutc,obs,obsm,choc,manv,id_vehicule_y,motor,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1,vma
0,200500000001,1.0,1,4,1,1.0,11.0,0.0,0.0,0.0,1976.0,A01,NaN,NaN,NaN,NaN,NaN,0.0,7,0.0,0.0,2.0,1.0,1.0,NaN,NaN,5,1,12,1900,3,2,1,1.0,3.0,11.0,CD41B,M,5051500.0,294400.0,590,3.0,41.0,0.0,B,2.0,2.0,1.0,430.0,0.0,1.0,1.0,0.0,63.0,1.0,0.0,1.0,0.0,NaN
1,200500000001,1.0,1,3,2,3.0,11.0,0.0,0.0,0.0,1968.0,B02,NaN,NaN,NaN,NaN,NaN,0.0,7,0.0,0.0,2.0,8.0,10.0,NaN,NaN,5,1,12,1900,3,2,1,1.0,3.0,11.0,CD41B,M,5051500.0,294400.0,590,3.0,41.0,0.0,B,2.0,2.0,1.0,430.0,0.0,1.0,1.0,0.0,63.0,1.0,0.0,1.0,0.0,NaN
2,200500000001,2.0,2,1,1,0.0,11.0,0.0,0.0,0.0,1964.0,B02,NaN,NaN,NaN,NaN,NaN,0.0,7,0.0,0.0,2.0,8.0,10.0,NaN,NaN,5,1,12,1900,3,2,1,1.0,3.0,11.0,CD41B,M,5051500.0,294400.0,590,3.0,41.0,0.0,B,2.0,2.0,1.0,430.0,0.0,1.0,1.0,0.0,63.0,1.0,0.0,1.0,0.0,NaN
3,200500000001,4.0,2,1,1,0.0,31.0,0.0,0.0,0.0,2004.0,B02,NaN,NaN,NaN,NaN,NaN,0.0,7,0.0,0.0,2.0,8.0,10.0,NaN,NaN,5,1,12,1900,3,2,1,1.0,3.0,11.0,CD41B,M,5051500.0,294400.0,590,3.0,41.0,0.0,B,2.0,2.0,1.0,430.0,0.0,1.0,1.0,0.0,63.0,1.0,0.0,1.0,0.0,NaN
4,200500000001,5.0,2,1,1,0.0,11.0,0.0,0.0,0.0,1998.0,B02,NaN,NaN,NaN,NaN,NaN,0.0,7,0.0,0.0,2.0,8.0,10.0,NaN,NaN,5,1,12,1900,3,2,1,1.0,3.0,11.0,CD41B,M,5051500.0,294400.0,590,3.0,41.0,0.0,B,2.0,2.0,1.0,430.0,0.0,1.0,1.0,0.0,63.0,1.0,0.0,1.0,0.0,NaN


In [6]:
df.shape

(2636489, 59)

In [7]:
print(df['an'].unique())

[   5    6    7    8    9   10   11   12   13   14   15   16   17   18
 2019 2020 2021 2022]


In [8]:
# Définir un dictionnaire de mapping pour convertir les valeurs de 'an'
mapping = {5: 2005, 6: 2006, 7: 2007, 8: 2008, 9: 2009, 10: 2010, 
           11: 2011, 12: 2012, 13: 2013, 14: 2014, 15: 2015, 
           16: 2016, 17: 2017, 18: 2018, 2019: 2019, 2020: 2020,
           2021: 2021, 2022: 2022}

# Appliquer la transformation en utilisant le mapping
df['an'] = df['an'].replace(mapping)

# Vérifier les valeurs uniques après transformation
print("Valeurs uniques pour la colonne 'an' après transformation :")
print(df['an'].unique())

Valeurs uniques pour la colonne 'an' après transformation :
[2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018
 2019 2020 2021 2022]


In [9]:
print(list(df['hrmn'].unique()))

[1900, 1600, 1845, 1615, 1945, 1415, 1530, 2115, 1715, 2015, 230, 615, 800, 1930, 2030, 2000, 730, 1630, 815, 1800, 1100, 900, 1030, 1130, 45, 2200, 1045, 715, 15, 2145, 1515, 500, 700, 2345, 1115, 2300, 1545, 1915, 1200, 600, 1815, 1215, 1345, 745, 30, 1000, 2215, 1830, 300, 1315, 2100, 830, 530, 945, 1645, 2045, 1330, 2315, 1015, 915, 1730, 1745, 1700, 845, 430, 1400, 645, 1230, 1145, 2245, 215, 930, 1500, 1300, 345, 445, 630, 330, 130, 2130, 515, 115, 1430, 400, 2330, 315, 1445, 100, 1245, 2230, 545, 415, 200, 145, 245, 1, 740, 1720, 1655, 1740, 1220, 1625, 120, 1840, 2040, 1835, 1925, 835, 825, 1735, 1355, 1435, 1150, 2105, 1210, 1505, 1754, 1020, 855, 950, 1525, 1405, 1340, 805, 1810, 1005, 1238, 1426, 1455, 2125, 1825, 1440, 1040, 50, 1950, 210, 944, 1059, 1534, 902, 750, 1407, 2010, 1414, 1607, 920, 942, 1425, 1755, 2058, 1935, 1726, 1532, 1725, 1617, 1520, 654, 1106, 1524, 10, 839, 236, 1436, 1302, 1453, 1947, 1152, 2250, 905, 1240, 1750, 1155, 1635, 2055, 1510, 1140, 650, 1125

In [10]:
# Définition de la fonction pour homogénéiser le format de l'heure
def homogenize_hour_format(row):
    # Convertir l'heure en chaîne de caractères
    hour_str = str(row['hrmn'])
    
    # Si l'année est comprise entre 2005 et 2018, ajuster le format de l'heure
    if row['an'] < 2019:
        # Extraire les deux derniers chiffres pour les minutes
        minutes = hour_str[-2:].zfill(2)
        
        # Les chiffres restants sont pour les heures
        hour = hour_str[:-2].zfill(2)
        
        # Concaténer avec ':' pour obtenir le format 'HH:MM'
        return f'{hour}:{minutes}'
    
    # Si l'année est 2019 ou plus, le format est déjà 'HH:MM'
    return hour_str

# Appliquer la fonction à la colonne 'hrmn'
df['hrmn'] = df.apply(homogenize_hour_format, axis=1)

In [11]:
print(df['hrmn'].unique())

['19:00' '16:00' '18:45' ... '02:27' '03:07' '00:00']


In [12]:
print(df['grav'].unique())

[ 4  3  1  2 -1]


In [13]:
# Suppression des lignes ou la target n'est pas renseignée
df = df.loc[df['grav'] != -1]

In [14]:
df.shape

(2636188, 59)

In [15]:
# Recherche de doublons
duplicates = df[df.duplicated()]

# Compte le nombre de lignes identiques
num_duplicates = len(duplicates)
print("Nombre de doublons trouvés :", num_duplicates)

Nombre de doublons trouvés : 2858


In [16]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [17]:
# Suppression des variables dites techniques
df.drop(columns=['Num_Acc', 'num_veh', 'id_vehicule_y', 'id_vehicule_x', 'id_usager'], inplace=True)

In [18]:
print(df['secu'].unique())

[11. 31. 21.  0. 12. 13. 90. 10. 22. 91. 33. 41. 23. 20. 92. 32. 43. 93.
 42.  1. nan 30. 40.  3.  2.]


In [19]:
# Remplacer les valeurs de 1 à 9 par NaN
df['secu'] = df['secu'].apply(lambda x: np.nan if 0 <= x <= 9 else x)

# Afficher les valeurs uniques après remplacement
print(df['secu'].unique())

[11. 31. 21. nan 12. 13. 90. 10. 22. 91. 33. 41. 23. 20. 92. 32. 43. 93.
 42. 30. 40.]


In [20]:
print(df['secu1'].unique())

[nan  1.  2.  8.  0.  3.  4.  5.  6.  9. -1.  7.]


In [21]:
filtered_df = df[(df['secu1'].isin([0, -1])) & (df[['secu2', 'secu3']].ne(-1).any(axis=1))]

# Afficher les résultats
filtered_df.head(5)

,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,secu1,secu2,secu3,senc,catv,occutc,obs,obsm,choc,manv,motor,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1,vma
2139543,1.0,1,3,1,5.0,NaN,-1.0,0,-1.0,1936.0,0.0,0.0,-1.0,2.0,7,NaN,0.0,2.0,3.0,15.0,1.0,2019,11,25,14:50,1,2,2,8.0,3.0,78126,DRIONNE (AVENUE DE LA),NaN,"48,8461250","2,1346200",78,3.0,321,0.0,NaN,2.0,4.0,3,605,0.0,2.0,1.0,NaN,NaN,2.0,5.0,1.0,NaN,50.0
2139565,10.0,3,4,2,5.0,NaN,3.0,3,1.0,1953.0,0.0,0.0,0.0,1.0,7,NaN,0.0,1.0,1.0,1.0,1.0,2019,11,28,16:35,1,2,1,1.0,6.0,66136,QUATRE-SAISONS (DES),NaN,"42,6865700","2,8939300",66,4.0,QUATRE-SAISONS (DES),0.0,NaN,2.0,2.0,0,0,0.0,1.0,1.0,NaN,NaN,1.0,0.0,1.0,NaN,50.0
2140200,1.0,1,3,1,5.0,NaN,-1.0,-1,-1.0,1967.0,0.0,6.0,-1.0,1.0,33,NaN,12.0,0.0,2.0,14.0,1.0,2019,8,14,16:05,1,2,1,1.0,6.0,63124,COSTES (RUE DIEUDONNE),NaN,"45,7454700","3,1673400",63,4.0,COSTES (RUE DIEUDONNE),0.0,NaN,2.0,2.0,(1),(1),0.0,1.0,1.0,NaN,NaN,1.0,0.0,1.0,NaN,50.0
2140218,10.0,3,3,2,0.0,NaN,1.0,3,1.0,1929.0,0.0,0.0,-1.0,3.0,1,NaN,0.0,1.0,1.0,15.0,5.0,2019,7,27,18:00,1,2,3,1.0,6.0,58194,ARDILLIERS,NaN,"46,9919700","3,1613000",58,4.0,ARDILLIERS,0.0,NaN,1.0,1.0,0,0,0.0,1.0,1.0,NaN,NaN,1.0,0.0,1.0,NaN,30.0
2140325,10.0,3,4,1,0.0,NaN,5.0,5,1.0,2004.0,0.0,0.0,-1.0,1.0,7,NaN,0.0,1.0,1.0,26.0,0.0,2019,6,13,17:20,1,2,1,2.0,6.0,93053,BREMENT (RUE DE) 109 A 235 - 110 A ...,NaN,"48,8866300","2,4716400",93,3.0,116,0.0,NaN,1.0,2.0,0,0,0.0,1.0,1.0,NaN,NaN,2.0,0.0,1.0,NaN,50.0


In [22]:
def transform_secu(value):
    if pd.isna(value):
        return np.nan
    value_str = str(int(value))  # Convert value to string to check the last digit
    last_digit = value_str[-1]
    if last_digit == '1':
        return int(value_str[0])  # Keep only the tens digit
    elif last_digit in ['2', '3', '0']:
        return 0
    return value

# Apply the function to create the new column
df['secu_all'] = df['secu'].apply(transform_secu)

# Display the DataFrame with the new column
df.head(5)
print(df['secu_all'].unique())

[ 1.  3.  2. nan  0.  9.  4.]


In [23]:
def combine_secu(row):
    # Extraire les valeurs de chaque colonne
    secu1, secu2, secu3 = row['secu1'], row['secu2'], row['secu3']
    
    # Si toutes les colonnes sont -1 ou NaN, alors NaN
    if (pd.isna(secu1) or secu1 == -1) and (pd.isna(secu2) or secu2 == -1) and (pd.isna(secu3) or secu3 == -1):
        return np.nan
    
    # Vérifier la priorité et la valeur à utiliser
    if secu1 > 0:
        return secu1
    elif secu2 > 0:
        return secu2
    elif secu3 > 0:
        return secu3
    
    # Sinon, si aucune valeur > 0 n'est trouvée, retourner 0
    return 0

# Appliquer la fonction pour créer la colonne secu_con
df['secu_con'] = df.apply(combine_secu, axis=1)
print(df['secu_con'].unique())

[nan  1.  2.  8.  0.  3.  4.  5.  6.  9.  7.]


In [24]:
df.tail()

,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,secu1,secu2,secu3,senc,catv,occutc,obs,obsm,choc,manv,motor,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1,vma,secu_all,secu_con
2633325,1.0,1,1,2,5.0,NaN,0.0,0,-1.0,2002.0,1.0,-1.0,-1.0,2.0,7,NaN,0.0,0.0,8.0,19.0,1.0,2022,1,1,08:40,1,1,3,1.0,3.0,81099,Chemin Toulze,NaN,"43,9272650000","1,9156370000",81,3.0,18,-1.0,D,2.0,2,30,125,0.0,1.0,1.0,NaN,-1,1.0,0.0,1.0,NaN,80.0,NaN,1.0
2633326,8.0,2,3,2,5.0,NaN,0.0,0,-1.0,2004.0,1.0,-1.0,-1.0,2.0,7,NaN,0.0,0.0,8.0,19.0,1.0,2022,1,1,08:40,1,1,3,1.0,3.0,81099,Chemin Toulze,NaN,"43,9272650000","1,9156370000",81,3.0,18,-1.0,D,2.0,2,30,125,0.0,1.0,1.0,NaN,-1,1.0,0.0,1.0,NaN,80.0,NaN,1.0
2633327,1.0,1,4,2,5.0,NaN,0.0,0,-1.0,1953.0,1.0,-1.0,-1.0,2.0,7,NaN,0.0,2.0,1.0,1.0,1.0,2022,1,1,08:40,1,1,3,1.0,3.0,81099,Chemin Toulze,NaN,"43,9272650000","1,9156370000",81,3.0,18,-1.0,D,2.0,2,30,125,0.0,1.0,1.0,NaN,-1,1.0,0.0,1.0,NaN,80.0,NaN,1.0
2633328,1.0,1,3,1,1.0,NaN,-1.0,-1,-1.0,1992.0,2.0,6.0,-1.0,1.0,33,NaN,0.0,2.0,1.0,1.0,1.0,2022,3,1,16:55,1,2,1,1.0,2.0,41018,RD956 - Pont Charles De Gaulle,NaN,"47,5944040000","1,3533290000",41,3.0,NaN,-1.0,NaN,3.0,4,(1),(1),0.0,1.0,1.0,NaN,4,1.0,0.0,1.0,NaN,70.0,NaN,2.0
2633329,1.0,1,1,1,-1.0,NaN,-1.0,-1,-1.0,2000.0,1.0,-1.0,-1.0,1.0,7,NaN,0.0,0.0,4.0,21.0,1.0,2022,3,1,16:55,1,2,1,1.0,2.0,41018,RD956 - Pont Charles De Gaulle,NaN,"47,5944040000","1,3533290000",41,3.0,NaN,-1.0,NaN,3.0,4,(1),(1),0.0,1.0,1.0,NaN,4,1.0,0.0,1.0,NaN,70.0,NaN,1.0


In [25]:
def merge_secu_columns(row):
    secu_all, secu_con = row['secu_all'], row['secu_con']
    
    # Retourner la première valeur non-NaN, sinon NaN
    if not pd.isna(secu_all):
        return secu_all
    if not pd.isna(secu_con):
        return secu_con
    return np.nan

# Appliquer la fonction pour créer la colonne secu_combined
df['secu_combined'] = df.apply(merge_secu_columns, axis=1)
print(df['secu_combined'].unique())

[ 1.  3.  2. nan  0.  9.  4.  8.  5.  6.  7.]


In [26]:
# Calculer le pourcentage de valeurs manquantes pour chaque colonne
missing_percentage = df.isnull().mean() * 100

# Afficher uniquement les colonnes avec des valeurs manquantes
missing_percentage = missing_percentage[missing_percentage > 0].round(2)

print(missing_percentage)

place             4.66
trajet            0.02
secu             24.00
locp              2.14
actp              2.14
etatp             2.14
an_nais           0.31
secu1            81.25
secu2            81.25
secu3            81.25
senc              0.01
occutc           18.52
obs               0.04
obsm              0.03
choc              0.01
manv              0.02
motor            81.25
atm               0.01
col               0.00
com               0.00
adr              13.11
gps              58.44
lat              40.46
long             40.46
catr              0.00
voie              8.90
v1               53.73
v2               95.03
circ              0.14
nbv               0.24
pr               38.74
pr1              38.88
vosp              0.24
prof              0.17
plan              0.21
lartpc           23.55
larrout           9.70
surf              0.17
infra             0.48
situ              0.44
env1             19.24
vma              81.25
secu_all         24.00
secu_con   

In [27]:
# Suppression des variables Ajout/Suppression 2019 après avoir combiné secu
df.drop(columns=['secu', 'secu1', 'secu2', 'secu3', 'motor', 'lartpc', 'env1', 'vma', 'secu_all', 'secu_con'], inplace=True)

In [28]:
# Suppression des variables à forte proportion de NaN et -1
df.drop(columns=['occutc', 'lat', 'long', 'v1', 'v2', 'pr', 'pr1', 'larrout','gps'], inplace=True)

# Calculer le pourcentage de valeurs manquantes pour chaque colonne
missing_percentage = df.isnull().mean() * 100

# Afficher uniquement les colonnes avec des valeurs manquantes
missing_percentage = missing_percentage[missing_percentage > 0].round(2)

print(missing_percentage)

place             4.66
trajet            0.02
locp              2.14
actp              2.14
etatp             2.14
an_nais           0.31
senc              0.01
obs               0.04
obsm              0.03
choc              0.01
manv              0.02
atm               0.01
col               0.00
com               0.00
adr              13.11
catr              0.00
voie              8.90
circ              0.14
nbv               0.24
vosp              0.24
prof              0.17
plan              0.21
surf              0.17
infra             0.48
situ              0.44
secu_combined     5.44
dtype: float64


In [29]:
# Suppression des variables jugées non pertinentes
df.drop(columns=['adr', 'com', 'dep', 'voie', 'nbv'], inplace=True)

In [30]:
df.duplicated().value_counts()

False    2632523
True         807
Name: count, dtype: int64

In [31]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [32]:
df.shape

(2632523, 33)

In [33]:
# Création d'un dictionnaire pour mapper les nouvelles valeurs de catv
new_catv_mapping = {
    1: [1, 50, 60, 80],
    2: [2, 4, 5, 30, 31, 32, 33, 34, 41, 42, 43],
    3: [7, 8, 9, 10, 11, 12],
    4: [13, 14, 15, 16, 17, 37, 38],
    5: [19, 20, 21, 35, 36, 39, 40, 99]
}
# Fonction pour mapper les nouvelles valeurs
def map_new_catv_value(old_value):
    for new_value, old_values in new_catv_mapping.items():
        if old_value in old_values:
            return new_value
    return None
# Supprimer la valeur 00
df = df[df['catv'] != 0]

# Appliquer la fonction de mapping pour créer les nouvelles valeurs
df['catv'] = df['catv'].apply(map_new_catv_value)

In [34]:
# Calculer le mode de la variable manv
mode_manv = df['manv'].mode()[0]

# Remplacer les valeurs -1 et 0 par le mode
df.loc[df['manv'].isin([-1, 0]), 'manv'] = mode_manv

# Création d'un dictionnaire pour mapper les nouvelles valeurs
new_manv_mapping = {
    1: list(range(1, 11)),
    2: [11, 12],
    3: [13, 14],
    4: [15, 16],
    5: [17, 18],
    6: list(range(19, 27))
}

# Fonction pour mapper les nouvelles valeurs
def map_new_manv_value(old_value):
    for new_value, old_values in new_manv_mapping.items():
        if old_value in old_values:
            return new_value
    return None

# Appliquer la fonction de mapping pour créer les nouvelles valeurs
df['manv'] = df['manv'].apply(map_new_manv_value)

In [35]:
df.duplicated().value_counts()

False    2631358
True         136
Name: count, dtype: int64

In [36]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [37]:
# Définir la fonction de transformation pour la variable obsm
def reduce_categories(x):
    if x == 0:
        return 1
    elif x == 1:
        return 2
    elif x == 2:
        return 3
    elif x in [4, 5, 6, 9]:
        return 4
    else:
        return np.nan

# Appliquer la transformation à la colonne 'obsm'
df['obsm'] = df['obsm'].apply(reduce_categories)

# Remplacer les valeurs manquantes par le mode le plus fréquent
mode_value = df['obsm'].mode()[0]
df['obsm'] = df['obsm'].fillna(mode_value)

In [38]:
locp_counts = df['obsm'].value_counts()
total_count = len(df['obsm'])
print("Valeurs uniques et leur décompte pour la variable 'obsm':")
for value, count in locp_counts.items():
    percentage = (count / total_count) * 100
    print(f"{value}: {count} ({percentage:.2f}%)")
    
na_count = df['obsm'].isna().sum()
percentage_missing = (na_count / total_count) * 100
print("Nombre de valeurs manquantes pour la variable 'obsm':", na_count)
print("Pourcentage de données manquantes pour la variable 'obsm': {:.2f}%".format(percentage_missing))

Valeurs uniques et leur décompte pour la variable 'obsm':
3.0: 1621903 (61.64%)
1.0: 557816 (21.20%)
2.0: 409303 (15.55%)
4.0: 42336 (1.61%)
Nombre de valeurs manquantes pour la variable 'obsm': 0
Pourcentage de données manquantes pour la variable 'obsm': 0.00%


In [39]:
# Définir la fonction de transformation pour la variable place
def reduce_categories(x):
    if x == 1:
        return 1
    elif x in [2, 3, 6]:
        return 2
    elif x in [4, 5, 7, 8, 9]:
        return 3
    elif x == 10:
        return 4
    else:
        return np.nan

# Appliquer la transformation à la colonne 'place'
df['place'] = df['place'].apply(reduce_categories)

# Remplacer les valeurs manquantes par le mode le plus fréquent
mode_value = df['place'].mode()[0]
df['place'] = df['place'].fillna(mode_value)

locp_counts = df['place'].value_counts()
total_count = len(df['place'])
print("Valeurs uniques et leur décompte pour la variable 'place':")
for value, count in locp_counts.items():
    percentage = (count / total_count) * 100
    print(f"{value}: {count} ({percentage:.2f}%)")
    
na_count = df['place'].isna().sum()
percentage_missing = (na_count / total_count) * 100
print("Nombre de valeurs manquantes pour la variable 'place':", na_count)
print("Pourcentage de données manquantes pour la variable 'place': {:.2f}%".format(percentage_missing))

Valeurs uniques et leur décompte pour la variable 'place':
1.0: 2144441 (81.50%)
2.0: 344341 (13.09%)
3.0: 104511 (3.97%)
4.0: 38065 (1.45%)
Nombre de valeurs manquantes pour la variable 'place': 0
Pourcentage de données manquantes pour la variable 'place': 0.00%


In [40]:
# Définir la fonction de transformation pour la variable int
def reduce_categories(x):
    if x == 1:
        return 1
    elif x in [2, 3, 4, 5, 8, 9]:
        return 2
    elif x == 6:
        return 3
    elif x == 7:
        return 4
    else:
        return np.nan

# Appliquer la transformation à la colonne 'int'
df['int'] = df['int'].apply(reduce_categories)

# Remplacer les valeurs manquantes par le mode le plus fréquent
mode_value = df['int'].mode()[0]
df['int'] = df['int'].fillna(mode_value)

locp_counts = df['int'].value_counts()
total_count = len(df['int'])
print("Valeurs uniques et leur décompte pour la variable 'int':")
for value, count in locp_counts.items():
    percentage = (count / total_count) * 100
    print(f"{value}: {count} ({percentage:.2f}%)")
    
na_count = df['int'].isna().sum()
percentage_missing = (na_count / total_count) * 100
print("Nombre de valeurs manquantes pour la variable 'int':", na_count)
print("Pourcentage de données manquantes pour la variable 'int': {:.2f}%".format(percentage_missing))

Valeurs uniques et leur décompte pour la variable 'int':
1.0: 1821552 (69.22%)
2.0: 715006 (27.17%)
3.0: 73841 (2.81%)
4.0: 20959 (0.80%)
Nombre de valeurs manquantes pour la variable 'int': 0
Pourcentage de données manquantes pour la variable 'int': 0.00%


In [41]:
# Variable situ
def reduce_categories(x):
    if x == 1:
        return 1
    elif x == 2:
        return 2
    elif x == 3:
        return 3
    elif x in [2, 5, 6, 8]:
        return 4
    else:
        return np.nan

# Appliquer la transformation à la colonne 'situ'
df['situ'] = df['situ'].apply(reduce_categories)

# Remplacer les valeurs manquantes par le mode le plus fréquent
mode_value = df['situ'].mode()[0]
df['situ'] = df['situ'].fillna(mode_value)

locp_counts = df['situ'].value_counts()
total_count = len(df['situ'])
print("Valeurs uniques et leur décompte pour la variable 'situ':")
for value, count in locp_counts.items():
    percentage = (count / total_count) * 100
    print(f"{value}: {count} ({percentage:.2f}%)")
    
na_count = df['situ'].isna().sum()
percentage_missing = (na_count / total_count) * 100
print("Nombre de valeurs manquantes pour la variable 'situ':", na_count)
print("Pourcentage de données manquantes pour la variable 'situ': {:.2f}%".format(percentage_missing))

Valeurs uniques et leur décompte pour la variable 'situ':
1.0: 2410459 (91.61%)
3.0: 164378 (6.25%)
4.0: 38139 (1.45%)
2.0: 18382 (0.70%)
Nombre de valeurs manquantes pour la variable 'situ': 0
Pourcentage de données manquantes pour la variable 'situ': 0.00%


In [42]:
# Définir la fonction de transformation de la variable choc
def reduce_categories(x):
    if x == 0:
        return 1
    elif x in [1, 2, 3]:
        return 2
    elif x in [4, 5, 6, 7, 8]:
        return 3
    elif x == 9:
        return 4
    else:
        return np.nan  
    
    
# Appliquer la transformation à la colonne 'choc'
df['choc'] = df['choc'].apply(reduce_categories)

mode_value = df['choc'].mode()[0]
df['choc'] = df['choc'].fillna(mode_value)

locp_counts = df['choc'].value_counts()
total_count = len(df['choc'])
print("Valeurs uniques et leur décompte pour la variable 'choc':")
for value, count in locp_counts.items():
    percentage = (count / total_count) * 100
    print(f"{value}: {count} ({percentage:.2f}%)")
    
na_count = df['choc'].isna().sum()
percentage_missing = (na_count / total_count) * 100
print("Nombre de valeurs manquantes pour la variable 'choc':", na_count)
print("Pourcentage de données manquantes pour la variable 'choc': {:.2f}%".format(percentage_missing))

Valeurs uniques et leur décompte pour la variable 'choc':
2.0: 1688420 (64.17%)
3.0: 742081 (28.20%)
1.0: 153668 (5.84%)
4.0: 47189 (1.79%)
Nombre de valeurs manquantes pour la variable 'choc': 0
Pourcentage de données manquantes pour la variable 'choc': 0.00%


In [43]:
# Variable atm
def reduce_categories(x):
    if x in [1, 8, 9]:
        return 1
    elif x in [2, 3, 4]:
        return 2
    elif x in [5, 7]:
        return 3
    elif x == 6:
        return 4
    else:
        return np.nan
    
# Appliquer la transformation à la colonne 'atm'
df['atm'] = df['atm'].apply(reduce_categories)


mode_value = df['atm'].mode()[0]
df['atm'] = df['atm'].fillna(mode_value)

locp_counts = df['atm'].value_counts()
total_count = len(df['atm'])
print("Valeurs uniques et leur décompte pour la variable 'atm':")
for value, count in locp_counts.items():
    percentage = (count / total_count) * 100
    print(f"{value}: {count} ({percentage:.2f}%)")
    
na_count = df['atm'].isna().sum()
percentage_missing = (na_count / total_count) * 100
print("Nombre de valeurs manquantes pour la variable 'atm':", na_count)
print("Pourcentage de données manquantes pour la variable 'atm': {:.2f}%".format(percentage_missing))

Valeurs uniques et leur décompte pour la variable 'atm':
1.0: 2229615 (84.73%)
2.0: 344931 (13.11%)
3.0: 50559 (1.92%)
4.0: 6253 (0.24%)
Nombre de valeurs manquantes pour la variable 'atm': 0
Pourcentage de données manquantes pour la variable 'atm': 0.00%


In [44]:
# Variable catr
def reduce_categories(x):
    if x == 1:
        return 1
    elif x == 2:
        return 2
    elif x == 3:
        return 3
    elif x == 4:
        return 4
    elif x in [5, 6, 7, 9]:
        return 5
    else:
        return np.nan
    
# Appliquer la transformation à la colonne 'catr'
df['catr'] = df['catr'].apply(reduce_categories)


mode_value = df['catr'].mode()[0]
df['catr'] = df['catr'].fillna(mode_value)

locp_counts = df['catr'].value_counts()
total_count = len(df['catr'])
print("Valeurs uniques et leur décompte pour la variable 'catr':")
for value, count in locp_counts.items():
    percentage = (count / total_count) * 100
    print(f"{value}: {count} ({percentage:.2f}%)")
    
na_count = df['catr'].isna().sum()
percentage_missing = (na_count / total_count) * 100
print("Nombre de valeurs manquantes pour la variable 'catr':", na_count)
print("Pourcentage de données manquantes pour la variable 'catr': {:.2f}%".format(percentage_missing))

Valeurs uniques et leur décompte pour la variable 'catr':
4.0: 1217578 (46.27%)
3.0: 871975 (33.14%)
1.0: 242857 (9.23%)
2.0: 222008 (8.44%)
5.0: 76940 (2.92%)
Nombre de valeurs manquantes pour la variable 'catr': 0
Pourcentage de données manquantes pour la variable 'catr': 0.00%


In [45]:
# Remplacer les valeurs manquantes par le mode le plus fréquent pour surf
mode_value = df['surf'].mode()[0]
df['surf'] = df['surf'].fillna(mode_value)

# Définir la fonction de transformation
def reduce_categories(x):
    if x == 1:
        return 1
    elif x in [2, 3, 4, 5, 6, 7, 8]:
        return 2
    elif x == 9:
        return 3
    else:
        return mode_value

# Appliquer la transformation à la colonne 'surf'
df['surf'] = df['surf'].apply(reduce_categories)
locp_counts = df['surf'].value_counts()
total_count = len(df['surf'])
print("Valeurs uniques et leur décompte pour la variable 'surf':")
for value, count in locp_counts.items():
    percentage = (count / total_count) * 100
    print(f"{value}: {count} ({percentage:.2f}%)")
    
na_count = df['surf'].isna().sum()
percentage_missing = (na_count / total_count) * 100
print("Nombre de valeurs manquantes pour la variable 'surf':", na_count)
print("Pourcentage de données manquantes pour la variable 'surf': {:.2f}%".format(percentage_missing))

Valeurs uniques et leur décompte pour la variable 'surf':
1.0: 2135457 (81.15%)
2.0: 484093 (18.40%)
3.0: 11808 (0.45%)
Nombre de valeurs manquantes pour la variable 'surf': 0
Pourcentage de données manquantes pour la variable 'surf': 0.00%


In [46]:
# Calculer le pourcentage de valeurs manquantes pour chaque colonne
missing_percentage = df.isnull().mean() * 100

# Afficher uniquement les colonnes avec des valeurs manquantes
missing_percentage = missing_percentage[missing_percentage > 0].round(2)

print(missing_percentage)
print(df.shape)

trajet           0.02
locp             2.14
actp             2.14
etatp            2.14
an_nais          0.29
senc             0.01
catv             0.58
obs              0.04
manv             0.02
col              0.00
circ             0.14
vosp             0.24
prof             0.17
plan             0.21
infra            0.48
secu_combined    5.43
dtype: float64
(2631358, 33)


In [47]:
df = df.dropna()
# Calculer le pourcentage de valeurs manquantes pour chaque colonne
missing_percentage = df.isnull().mean() * 100

# Afficher uniquement les colonnes avec des valeurs manquantes
missing_percentage = missing_percentage[missing_percentage > 0].round(2)

print(missing_percentage)
print(df.shape)

Series([], dtype: float64)
(2402001, 33)


In [48]:
# Suppression des variables présentant une catégorie dominante qui correspond a "non renseigné"
df.drop(columns=['actp', 'etatp', 'locp', 'obs', 'senc', 'trajet', 'vosp'], inplace=True)

In [49]:
df.duplicated().value_counts()

False    2398108
True        3893
Name: count, dtype: int64

In [50]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [51]:
# Calculer la fréquence de chaque valeur
value_counts = df['secu_combined'].value_counts()

# Calculer le pourcentage pour chaque valeur
value_percentages = (value_counts / len(df)) * 100

# Convertir en DataFrame pour un affichage plus structuré
percentages_df = value_percentages.reset_index()
percentages_df.columns = ['secu_combined', 'Percentage']

# Afficher le résultat
print(percentages_df)

   secu_combined  Percentage
0            1.0   59.545483
1            2.0   19.400169
2            0.0   17.455969
3            8.0    2.399558
4            3.0    0.725197
5            9.0    0.329134
6            4.0    0.117509
7            6.0    0.017097
8            5.0    0.009257
9            7.0    0.000625


In [52]:
# transformation de secu_combined
def reduce_categories(x):
    if x == 0:
        return 1
    elif x == 1:
        return 2
    elif x == 2:
        return 3
    elif x in [3, 4, 5, 6, 7,8, 9]:
        return 4
    else:
        return np.nan
    
# Appliquer la transformation à la colonne 'secu_combined'
df['secu_combined'] = df['secu_combined'].apply(reduce_categories)
locp_counts = df['secu_combined'].value_counts()
total_count = len(df['secu_combined'])
print("Valeurs uniques et leur décompte pour la variable 'secu_combined':")
for value, count in locp_counts.items():
    percentage = (count / total_count) * 100
    print(f"{value}: {count} ({percentage:.2f}%)")
    
na_count = df['secu_combined'].isna().sum()
percentage_missing = (na_count / total_count) * 100
print("Nombre de valeurs manquantes pour la variable 'secu_combined':", na_count)
print("Pourcentage de données manquantes pour la variable 'secu_combined': {:.2f}%".format(percentage_missing))

Valeurs uniques et leur décompte pour la variable 'secu_combined':
2: 1427965 (59.55%)
3: 465237 (19.40%)
1: 418613 (17.46%)
4: 86293 (3.60%)
Nombre de valeurs manquantes pour la variable 'secu_combined': 0
Pourcentage de données manquantes pour la variable 'secu_combined': 0.00%


In [53]:
df = df.replace(-1, np.nan)

In [54]:
# Calculer le pourcentage de valeurs manquantes pour chaque colonne
missing_percentage = df.isnull().mean() * 100

# Afficher uniquement les colonnes avec des valeurs manquantes
missing_percentage = missing_percentage[missing_percentage > 0].round(2)

print(missing_percentage)
print(df.shape)

sexe     0.00
lum      0.00
col      0.16
circ     1.06
prof     0.00
plan     0.00
infra    0.18
dtype: float64
(2398108, 26)


In [55]:
df = df.dropna()
print(df.shape)

(2364765, 26)


In [56]:
df.duplicated().value_counts()

False    2364764
True           1
Name: count, dtype: int64

In [57]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [58]:
# Créer une variable catégorielle découpant les heures par tranche d'une heure
df['hour_cat'] = pd.to_datetime(df['hrmn'], format='%H:%M').dt.hour
print(df['hrmn'].unique())
print(df['hour_cat'].unique())

['19:00' '16:00' '18:45' ... '02:27' '03:07' '00:00']
[19 16 18 14 15 21 17 20  2  6  8  7 11 10  0  9 22  5 23 12 13  3  4  1]


In [59]:
mode_value = df['hour_cat'].mode()[0]
def reduce_categories(x):
    if x in range(8, 17):
        return 1
    elif (x in range(6, 9)) or (x in range(16, 19)):
        return 2
    elif (x in range(21, 25)) or (x in range(1, 6)):
        return 3
    else:
        return mode_value

# Appliquer la transformation à la colonne 'hour_cat'
df['hour_cat'] = df['hour_cat'].apply(reduce_categories)

In [60]:
# Calculer l'âge de l'usager au moment de l'accident en soustrayant l'année de naissance de l'année de l'accident
df['age_usag'] = df['an'] - df['an_nais']

# Afficher les premières lignes du DataFrame pour vérifier
print(df[['an_nais', 'an', 'age_usag']].head())


class AgeCat(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        return None
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        return pd.DataFrame(pd.cut(X['age_usag'], bins = [0, 12, 18, 30, 58, np.max(df.age_usag)], labels=['Enfant','Adolescent',
                                                                                                    'Jeune adulte','Adulte',
                                                                                                    'Senior']))


# Créer un transformateur personnalisé pour binariser l'âge des usagers
age_categorized = AgeCat()

# Appliquer le transformateur sur les données
df['age_category'] = age_categorized.fit_transform(df[['age_usag']])

# Afficher les premières lignes du DataFrame avec la nouvelle variable
print(df[['age_usag', 'age_category']].head())

   an_nais    an  age_usag
0   1976.0  2005      29.0
1   1968.0  2005      37.0
2   1964.0  2005      41.0
3   2004.0  2005       1.0
4   1998.0  2005       7.0
   age_usag  age_category
0      29.0  Jeune adulte
1      37.0        Adulte
2      41.0        Adulte
3       1.0        Enfant
4       7.0        Enfant


In [61]:
def jour_de_la_semaine(date):
    jours = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"]
    return jours[date.weekday()]

df["hrmn"] = df["hrmn"].apply(lambda x: str(x).zfill(4))

df["datetime_str"] = (
    df["an"].astype(str)
    + "-"
    + df["mois"].astype(str)
    + "-"
    + df["jour"].astype(str)
    + " "
    + df["hrmn"]
)
df["datetime"] = pd.to_datetime(df["datetime_str"], format="%Y-%m-%d %H:%M")
df["jour_sem"] = df["datetime"].apply(jour_de_la_semaine)
# + clear cols temp
df["jour_sem"].unique()

array(['Mercredi', 'Vendredi', 'Mardi', 'Lundi', 'Samedi', 'Dimanche',
       'Jeudi'], dtype=object)

In [62]:
# Créer une instance de LabelEncoder pour 'age_category'
label_encoder_age_category = LabelEncoder()
df['age_category_encoded'] = label_encoder_age_category.fit_transform(df['age_category'])

# Créer une instance de LabelEncoder pour 'jour_sem'
label_encoder_jour_sem = LabelEncoder()
df['jour_sem_encoded'] = label_encoder_jour_sem.fit_transform(df['jour_sem'])

In [63]:
# Afficher un aperçu des colonnes 'age_category' et 'age_category_encoded'
print(df[['age_category', 'age_category_encoded']].head())

print(df[['jour_sem', 'jour_sem_encoded']].head())

   age_category  age_category_encoded
0  Jeune adulte                     3
1        Adulte                     1
2        Adulte                     1
3        Enfant                     2
4        Enfant                     2
   jour_sem  jour_sem_encoded
0  Mercredi                 4
1  Mercredi                 4
2  Mercredi                 4
3  Mercredi                 4
4  Mercredi                 4


In [64]:
# Suppression des variables "Temps" du DataFrame df.
df.drop(columns=['an', 'an_nais', 'jour', 'mois', 'hrmn', 'age_usag', 'datetime_str', 'datetime', 'age_category', 'jour_sem'], inplace=True)

In [65]:
df.shape

(2364764, 24)

In [66]:
df.head()

,place,catu,grav,sexe,catv,obsm,choc,manv,lum,agg,int,atm,col,catr,circ,prof,plan,surf,infra,situ,secu_combined,hour_cat,age_category_encoded,jour_sem_encoded
0,1.0,1,4,1.0,3.0,3.0,2.0,1.0,3.0,2,1.0,1.0,3.0,3.0,2.0,1.0,1.0,1.0,0.0,1.0,2,17,3,4
1,1.0,1,3,2.0,3.0,3.0,3.0,1.0,3.0,2,1.0,1.0,3.0,3.0,2.0,1.0,1.0,1.0,0.0,1.0,2,17,1,4
2,2.0,2,1,1.0,3.0,3.0,3.0,1.0,3.0,2,1.0,1.0,3.0,3.0,2.0,1.0,1.0,1.0,0.0,1.0,2,17,1,4
3,3.0,2,1,1.0,3.0,3.0,3.0,1.0,3.0,2,1.0,1.0,3.0,3.0,2.0,1.0,1.0,1.0,0.0,1.0,4,17,2,4
4,3.0,2,1,1.0,3.0,3.0,3.0,1.0,3.0,2,1.0,1.0,3.0,3.0,2.0,1.0,1.0,1.0,0.0,1.0,2,17,2,4


In [67]:
df = df.astype(int)

In [68]:
df.duplicated().value_counts()

False    1810544
True      554220
Name: count, dtype: int64

In [69]:
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [70]:
df.shape

(1810544, 24)

In [71]:
df.head()

,place,catu,grav,sexe,catv,obsm,choc,manv,lum,agg,int,atm,col,catr,circ,prof,plan,surf,infra,situ,secu_combined,hour_cat,age_category_encoded,jour_sem_encoded
0,1,1,4,1,3,3,2,1,3,2,1,1,3,3,2,1,1,1,0,1,2,17,3,4
1,1,1,3,2,3,3,3,1,3,2,1,1,3,3,2,1,1,1,0,1,2,17,1,4
2,2,2,1,1,3,3,3,1,3,2,1,1,3,3,2,1,1,1,0,1,2,17,1,4
3,3,2,1,1,3,3,3,1,3,2,1,1,3,3,2,1,1,1,0,1,4,17,2,4
4,3,2,1,1,3,3,3,1,3,2,1,1,3,3,2,1,1,1,0,1,2,17,2,4


In [72]:
df.to_csv(os.path.join(output_path, 'Processed_df.csv'), index=False)

SelectKBest - Chi²

In [73]:
# Diviser les données en fonction des caractéristiques (X) et de la cible (y)
X = df.drop(columns=['grav'])  # Caractéristiques
y = df['grav']  # Cible

In [74]:
k_best = SelectKBest(chi2, k=23)
k_best.fit_transform(X, y)

array([[ 1,  1,  1, ..., 17,  3,  4],
       [ 1,  1,  2, ..., 17,  1,  4],
       [ 2,  2,  1, ..., 17,  1,  4],
       ...,
       [ 3,  2,  2, ...,  1,  0,  5],
       [ 1,  1,  1, ...,  1,  3,  3],
       [ 1,  1,  1, ...,  1,  3,  3]])

In [75]:
# Créer un DataFrame pour stocker les résultats du test chi²
chi2_results = pd.DataFrame({'Variable': X.columns, 'Chi2 Score': k_best.scores_})

# Trier les résultats par score chi² de manière décroissante
chi2_results_sorted = chi2_results.sort_values(by='Chi2 Score', ascending=False)

# Calculer la somme des scores chi² de toutes les variables
total_chi2_score = chi2_results_sorted['Chi2 Score'].sum()

# Créer une colonne pour stocker la proportion de chaque variable
chi2_results_sorted['Proportion'] = (chi2_results_sorted['Chi2 Score'] / total_chi2_score) * 100

# Afficher les résultats avec la nouvelle colonne
print(chi2_results_sorted)

                Variable    Chi2 Score  Proportion
3                   catv  29369.201164   18.610780
4                   obsm  15744.893362    9.977280
1                   catu  13237.630911    8.388469
18                  situ  12839.441321    8.136143
11                   col  12114.328161    7.676651
6                   manv  11857.903831    7.514159
15                  plan  10658.346955    6.754019
8                    agg  10350.948887    6.559226
0                  place   8515.320725    5.396019
19         secu_combined   8271.096196    5.241258
17                 infra   7536.360742    4.775668
9                    int   4534.069159    2.873165
21  age_category_encoded   3585.692851    2.272195
2                   sexe   3042.447312    1.927949
12                  catr   2424.324476    1.536255
7                    lum   1436.764018    0.910454
13                  circ    717.285808    0.454532
22      jour_sem_encoded    429.443943    0.272132
14                  prof    370

In [76]:
# Liste des variables à conserver = 16 variables
variables_a_garder = ['catu', 'catv', 'obsm', 'col', 'place', 'manv', 'situ', 'agg', 'plan', 'secu_combined', 'age_category_encoded', 'infra', 'int', 'sexe','catr', 'lum']

# Garder seulement les colonnes spécifiées
df_16 = df.filter(variables_a_garder)
df_16.shape

(1810544, 16)

In [77]:
df_16 = df_16.astype('object')
# Créer un nouveau DataFrame avec les variables catégorielles encodées en one-hot

df_encoded_16 = pd.get_dummies(df_16, drop_first = True)

In [80]:
X = df_encoded_16

# Appliquer l'undersampling sur X et y en spécifiant un random_state
undersample = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersample.fit_resample(X, y)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [81]:
# Exporter les données d'entraînement et de test en CSV

datasets = {'train': (X_train, y_train), 'test': (X_test, y_test)}

for dataset_name, (X, y) in datasets.items():

    # Convertir X et y en DataFrames pandas

    X_df = pd.DataFrame(X)

    y_df = pd.DataFrame(y)

    # Concaténer X et y en un seul DataFrame

    dataset_df = pd.concat([X_df, y_df], axis=1)
    
    dataset_df.to_csv(os.path.join(output_path, f"{dataset_name}_data16variables.csv"), index=False)